## Read and plot near-real-time Wave Glider data

first cut by Tom, 10/18/2021  
Updated for IOP1, 10/9/2022

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' 
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-127,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,21)-2.5
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 41)
    levels = np.linspace(13,18,11)

    

Payload 2 Table 1 has met, ctd variables  
Payload 2 Table 2 has RDI variables

In [4]:
# List of WGs
input_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'KELVIN', 'CARSON']
url_prefix = 'http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/'
tab1_postfix = '_21.nc'#PLD2_TAB1
tab2_postfix = '_22.nc' #PLD2_TAB2
position_postfix = '_23.nc' # position??
WG_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'KELVIN', 'CARSON']
outpath='../data/raw/WG_NRT_IOP2/'

#http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/KELVIN_23.nc.html
#http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/KELVIN_23.nc

In [5]:
# For some reason, reading the files over the internet directly is not working well
# Download instead

n=0
file_list1 = []
file_list2 = []
file_list3 = []
for WG in WG_list:
    input_WG=input_list[n]
    outfile1 = outpath+input_WG+tab1_postfix
    outfile2 = outpath+input_WG+tab2_postfix
    outfile3 = outpath+input_WG+position_postfix
    # Read and save table 1 files
    url1 = url_prefix+input_WG+tab1_postfix
    file_data = requests.get(url1).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile1, "wb") as file:
        file.write(file_data)
    # Read and save table 2 files
    url2 = url_prefix+input_WG+tab2_postfix
    file_data = requests.get(url2).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile2, "wb") as file:
        file.write(file_data)
    # Read and save position files
    url3 = url_prefix+input_WG+position_postfix
    file_data = requests.get(url3).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile3, "wb") as file:
        file.write(file_data)
    n=n+1
    print(url3)
    file_list1.append(outfile1)
    file_list2.append(outfile2)
    file_list3.append(outfile3)


http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/WHOI22_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/WHOI32_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/WHOI43_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/STOKES_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/PLANCK_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/KELVIN_23.nc
http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/CARSON_23.nc


In [6]:
file_list2

['../data/raw/WG_NRT_IOP2/WHOI22_22.nc',
 '../data/raw/WG_NRT_IOP2/WHOI32_22.nc',
 '../data/raw/WG_NRT_IOP2/WHOI43_22.nc',
 '../data/raw/WG_NRT_IOP2/STOKES_22.nc',
 '../data/raw/WG_NRT_IOP2/PLANCK_22.nc',
 '../data/raw/WG_NRT_IOP2/KELVIN_22.nc',
 '../data/raw/WG_NRT_IOP2/CARSON_22.nc']

In [7]:
def fix_ds_time(ds):
    '''
    Drop nonunique values in realtime data files and sort time.
    
    Input: ds, xarray dataset
    Output: ds, xarray dataset
    '''
    t, ind = np.unique(ds.time, return_index=True)
    ds2 = ds.isel(time=ind,drop=True)
    return ds2

In [8]:
# Read in files (Payload 2 Tables 1 and 2; position) from all WG
n=0
for WG in WG_list:
    input_WG=input_list[n]
    file1 = file_list1[n]
    file2 = file_list2[n]
    file3 = file_list3[n]
    varstr = 'met_'+WG
    ds_met_temp=xr.open_dataset(file1,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_met_temp) #Drop nonunique values and sort time
    varstr = 'adcp_'+WG
    !ncrename -v z,z_matrix $file2 #renaming variable z to prevent dimension/variable name conflict in xarray, requires nco in linux
    ds_adcp_temp=xr.open_dataset(file2,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_adcp_temp) #Drop nonunique values and sort time
    varstr = 'pos_'+WG
    ds_pos_temp=xr.open_dataset(file3,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_pos_temp) #Drop nonunique values and sort time
    n=n+1
    print(file1)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy', 'pydap']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

In [ ]:
eval('met_'+WG)

In [ ]:
# Write WHOI22 met record to file
# met_WHOI22.to_netcdf('../data/raw/WG_NRT/WHOI22_met.nc')


In [ ]:
# Now we can access these in a loop using syntax like:
# eval('adcp_'+WG_list[7])

In [ ]:
eval('met_'+WG_list[0])

In [ ]:
#Compute density from T and cond
p = 1
for WG in WG_list:
    ds = eval('met_'+WG)
    ds['uctd_psu_Avg']=gsw.conversions.SP_from_C(10*ds.uctd_cond_Avg, ds.uctd_temp_Avg, p)
    SA = gsw.conversions.SA_from_SP(ds.uctd_psu_Avg, 1,ds.longitude_1hz_Avg, ds.latitude_1hz_Avg)
    CT = gsw.conversions.CT_from_t(SA, ds.uctd_temp_Avg, p)
    ds['uctd_sigma0_Avg'] = gsw.density.sigma0(SA, CT)
    varstr = 'met_'+WG
    locals()[varstr]= ds

In [ ]:
def plot_WG_SST(V,n,tmin):
    extent = [xmin, xmax, ymin, ymax]
    ds = eval('met_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    sst = ds2.uctd_temp_Avg.values.astype(np.ndarray)
    ax.set_title('WG SST',size = 10.)
    plt.scatter(ds2.longitude_1hz_Avg, ds2.latitude_1hz_Avg,s=5,c=sst, cmap=plt.get_cmap('turbo'),vmin=V[0],vmax=V[1],transform=ccrs.PlateCarree())
    
    

In [ ]:
def plot_WG_sigma0(V,n,tmin):
    extent = [xmin, xmax, ymin, ymax]
    ds = eval('met_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    rho = ds2.uctd_sigma0_Avg.values.astype(np.ndarray)
    ax.set_title('$\sigma_0$',size = 10.)
    plt.scatter(ds2.longitude_1hz_Avg, ds2.latitude_1hz_Avg,s=5,c=rho, cmap=plt.get_cmap('turbo'),vmin=V[0],vmax=V[1],transform=ccrs.PlateCarree())
 

In [ ]:
fig = plt.figure()
V = [14,18]
Vrho = [23,24.5]
tmin = np.datetime64('2022-10-10T00:00:00')
n = 0
extent = [xmin, xmax, ymin, ymax]
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
ax.set_extent(extent, crs=ccrs.PlateCarree())


for n in range(8):
        #plot_WG_time(n)
        #plot_WG_SST(V,n,tmin)
        plot_WG_sigma0(Vrho,n,tmin)


plt.colorbar(location = 'bottom')
ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')

gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
functions.plot_ops_area(ax,transform=ccrs.PlateCarree(),color='k');


In [ ]:
def plot_WG_sigma0_lat(n,tmin):
    ds = eval('met_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    rho = ds2.uctd_sigma0_Avg.values.astype(np.ndarray)
    ax.set_title('$\sigma_0$',size = 10.)
    plt.scatter(rho,ds2.latitude_1hz_Avg,s=5,c=(ds2.time-tmin)*10**-9/(60*60*24), cmap=plt.get_cmap('turbo'))


In [ ]:
fig = plt.figure()
V = [14,18]
Vrho = [23,24.5]
tmin = np.datetime64('2022-10-09T00:00:00')
for n in range(8):
        #plot_WG_time(n)
        #plot_WG_SST(V,n,tmin)
        plot_WG_sigma0_lat(n,tmin)

plt.colorbar(location = 'right')


OK, now let's look at RDI files (Table 2)

In [ ]:
# eval('adcp_'+WG_list[7])

In [ ]:
# eval('met_'+WG_list[7])

In [ ]:
# eval('pos_'+WG_list[7])

OK, we have 15 minute files from the ADCP and 5 minute from the position files.  Interpolate the position files to the ADCP times.  That should be easy using xarray interp package, following:  
https://docs.xarray.dev/en/stable/user-guide/interpolation.htmlhttps://docs.xarray.dev/en/stable/user-guide/interpolation.html  

```
new_lon = -126.1
new_lat = 37.1
new_time = ds.time[-3]
dsi = ds.interp(time=new_time,latitude=new_lat, longitude=new_lon)
```

```
new_time = ds_adcp.time
ds_pos_i = ds_pos.interp(time=new_time)
```

In [ ]:
# ds_adcp = eval('adcp_'+WG_list[7])
# ds_pos = eval('pos_'+WG_list[7])

In [ ]:
# new_time = np.unique(ds_adcp.time)
# ds_pos_i = ds_pos.interp(time=new_time)

In [ ]:
eval('adcp_'+WG_list[2])

In [ ]:
# Interpolate each WG's position to ADCP time and add to ADCP file
for WG in WG_list:
    ds_adcp = eval('adcp_'+WG)
    ds_pos = eval('pos_'+WG)
    ds_pos_i = ds_pos.interp(time=ds_adcp.time)
    ds_adcp['Longitude']=ds_pos_i.Longitude
    ds_adcp['Latitude']=ds_pos_i.Latitude
    varstr = 'adcp_'+WG
    locals()[varstr]= ds_adcp
    del ds_adcp

In [ ]:
eval('adcp_'+WG_list[2])

In [ ]:
tmin = np.datetime64('2022-10-10T00:00:00')
tmax = np.datetime64('now')
vmin = -.50 
vmax = .50
levels=np.arange(vmin,vmax,.05)


In [ ]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(8,1,8)
ax1.set_xlim(tmin,tmax)
zmax=-60
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ax = plt.subplot(8,1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_east,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(zmax, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('East vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)

In [ ]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(8,1,8)
ax1.set_xlim(tmin,tmax)
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ax = plt.subplot(8,1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_north,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(-60, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('North vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)

OK, that's very cool!  I have all the files cleaned up and have added the lat/lon.  Le't get ready to try finding the ones in the tringle and doing the LS fit.  Maybe a good intermediate step is to plot the vectors on a map.  Or, maybe better would be to do the same plots as the last two above, but showing only the data from the triangle.

In [ ]:
lon0 = -124.66
lat0 = 36.96
tol = .023


In [ ]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(8,1,8)
ax1.set_xlim(tmin,tmax)
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    ax = plt.subplot(8,1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_north,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(-60, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('North vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
if savefig:
    plt.savefig(__figdir__+'WG_triangle_north_vel'+'.'+plotfiletype,**savefig_args)

In [ ]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(8,1,8)
ax1.set_xlim(tmin,tmax)
zmax=-60
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    ax = plt.subplot(8,1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_east,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(zmax, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('East vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
if savefig:
    plt.savefig(__figdir__+'WG_triangle_east_vel'+'.'+plotfiletype,**savefig_args)

OK, WHOI32 has been in the first triangle for about the whole time (since the 12th).  Let's plot a time series of that one.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
WG = 'WHOI32'
ds = eval('adcp_'+WG)
ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_east,vmin=vmin,vmax=vmax)
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
plt.ylim(zmax, 0)
plt.text(tmin,zmax+5,WG)
plt.title('East vel for ' + WG + ' when in triangle')
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
ax.set_xlim(tmin,tmax)
if 0: #savefig:
    plt.savefig(__figdir__+'WHOI32_triangle_east_vel'+'.'+plotfiletype,**savefig_args)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
WG = 'WHOI32'
ds = eval('adcp_'+WG)
ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
ind=np.flatnonzero(np.isnan(ds.z_matrix[1][:])==False)
z = ds.z_matrix[:,ind[0]]
z0 = -15
zind = np.flatnonzero(np.abs(z-z0)<1)
plt.plot(ds.time.values,np.squeeze(ds.current_east[zind,]))
plt.plot(ds.time.values,np.squeeze(ds.current_north[zind,]))
plt.legend(['U','V'])
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
plt.title(WG+ ' east vel')
plt.ylabel('[m/s]')
fig.autofmt_xdate()
ax.set_xlim(tmin,tmax)
if savefig:
    plt.savefig(__figdir__+'WHOI32_triangle_time_series'+'.'+plotfiletype,**savefig_args)

In [ ]:
9*60*60*.15

In [ ]:
%whos

Consider the tapered and weighted least-squares solution (Equation 1.125 in the course notes),
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{E}+\mathbf{S}^{-1}\right)^{-1}\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{y}+\mathbf{S}^{-1}\mathbf{x_0}\right).
 \label{TW_LS}
\end{equation}
Recall that $\mathbf{W}^{-1}$ is a ''weight matrix'', $\mathbf{S}^{-1}$ is a ''taper matrix'' (which can be thought of as another weight matrix, as we shall see soon), and $\mathbf{x_0}$ is the first guess solution.  Just to simplify the notation and discussion a little bit, we will assume that $\mathbf{x_0}=0$, which would be the case if we know or think that the expectation value $<\mathbf{x}>=0$.  In that case
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{E}+\mathbf{S}^{-1}\right)^{-1}\mathbf{E}^T\mathbf{W}^{-1}\mathbf{y}.
 \label{TW_LS2}
\end{equation}

Again assuming $\mathbf{x_0}=0$, the cost function that led to Equation \ref{TW_LS2} was (Equation 1.193 in the notes):
\begin{equation}
  J=\mathbf{n}^T \mathbf{W}^{-1}\mathbf{n}+\mathbf{x}^T\mathbf{S}^{-1}\mathbf{x}.
 \label{J_TW_LS2}
\end{equation}

Like most complicated equations, we can get a better feel for what the equation means by considering some special cases.  A common special case to consider in matrix problems is one where some matrices are diagonal and square, because these matrices can easily be inverted.  If $\mathbf{W}=a \mathbf{I}$, then $\mathbf{W}^{-1}=\frac{1}{a} \mathbf{I}$. So, let's try letting $\mathbf{W}^{-1}=\frac{1}{\sigma_n^2} \mathbf{I}$ and letting $\mathbf{S}^{-1}=\frac{1}{\Delta_x^2} \mathbf{I}$.  Then, the cost function in Equation \ref{J_TW_LS2} becomes
\begin{equation}
  J=\frac{1}{\sigma_n^2}\mathbf{n}^T \mathbf{n}+\frac{1}{\Delta_x^2}\mathbf{x}^T \mathbf{x},
 \label{J_TW_LS2_simple}
\end{equation}
and Equation \ref{TW_LS2} becomes:
\begin{equation}
  \tilde{\mathbf{x}}=\left(\frac{1}{\sigma_n^2}\mathbf{E}^T \mathbf{E}+\frac{1}{\Delta_x^2}\mathbf{I}\right)^{-1}\frac{1}{\sigma_n^2}\mathbf{E}^T\ \mathbf{y},
  \nonumber
\end{equation}
or,
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{E}+\frac{\sigma_n^2}{\Delta_x^2}\mathbf{I}\right)^{-1}\mathbf{E}^T \mathbf{y}.
 \label{TW_LS2_simple}
\end{equation}
If $\sigma_n^2$ is the expected noise variance and $\Delta_x^2$ is the expected solution variance, then we can interpret Equation \ref{J_TW_LS2_simple} as a cost function where we equally penalize (in a normalized sense) the estimated noise variance and the estimated solution variance.  We are simultaneously minimizing the model-data misfit and the solution variance.





The tapering parameter $\sigma_n^2/\Delta_x^2$ can be considered to be an inverse signal-to-noise ratio (SNR), expressing our expectation about the relative variance of the measurement noise and the solution.  In the limit that the tapering parameter is very small (meaning the SNR is high), Equation \ref{TW_LS2_simple} is just the ordinary least squares solution.  If the tapering parameter is small, the tapered least squares solution could also be viewed as a mere computational trick-- by adding a small value to the diagonal of $\mathbf{E}^T\mathbf{E}$, we have guaranteed that the inverse $\left(\mathbf{E}^T\mathbf{E}+\frac{\sigma_n^2}{\Delta_x^2}\mathbf{I}\right)^{-1}$ exists.
